In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from scipy.ndimage import convolve
import math
import os
import random

import os, os.path
from random import sample
import shutil

In [2]:
def showimg(img,name):
    print(img.shape)
    plt.imshow(img)
    plt.title(name)
    plt.show()

In [3]:
def colorRGB(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

colors=['#808000','#00FF00','#0000FF','#FFFF00','#00FFFF','#FF00FF','#808080','#FF8080','#80FF80','#8080FF','#008080','#800080','#000000','#FFFFFF','#FF0000']
coice = [1,2,3,4,5,6,7,8,9,10,11,12,127,255,127,60,127,60]
rgbcols=[]
for i in colors:
    rgbcols.append(colorRGB(i))
colmap={}
j=0
for i in range(len(coice)):
    if(coice[i] not in colmap):
        colmap[coice[i]]=(rgbcols[j][2],rgbcols[j][1],rgbcols[j][0])
        j+=1
# colmap
def transformtotarget(impath,imname,imsavepath):
    img = cv2.imread(impath+imname)
#     showimg(img,"nn")
    c1,c2,c3 = cv2.split(img)
    conds = [c2==0,c2==1,c2==2,c2==3,c2==4,c2==5,c2==6,c2==7,c2==8,c2==9,c2==10,c2==11,c2==12,c2==13,c2==14,c2==15,c2==16,c2==17]
    coice = [1,2,3,4,5,6,7,8,9,10,11,12,127,255,127,60,127,60]
    mb=np.select(conds,coice,default=0)
    gtmap = np.zeros((256,256,3), np.uint8)
    for k,v in colmap.items():
        gtmap[np.where(mb==k)]=v
#     showimg(mb,"plot")
    cv2.imwrite(imsavepath+imname,gtmap)

In [6]:
colmap # pixelvalue to color mapping: colors in 'BGR' order

{1: (0, 128, 128),
 2: (0, 255, 0),
 3: (255, 0, 0),
 4: (0, 255, 255),
 5: (255, 255, 0),
 6: (255, 0, 255),
 7: (128, 128, 128),
 8: (128, 128, 255),
 9: (128, 255, 128),
 10: (255, 128, 128),
 11: (128, 128, 0),
 12: (128, 0, 128),
 127: (0, 0, 0),
 255: (255, 255, 255),
 60: (0, 0, 255)}

# Generating ground truth/target data

In [ ]:
# impath="/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/floorplan_dataset/"
# imsavepath="/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/train/target/"
impath="PATH_TO_RPLAN_DATASET"
imsavepath = "PATH_TO_SAVE_DIR"
imglst=os.listdir(impath)
print(len(imglst))

In [ ]:
for i in range(len(imglst)):
    transformtotarget(impath,imglst[i],imsavepath)

# Generating inputs

In [ ]:
c1,c2,c3 = cv2.split(img)
npu=np.unique(c2)
npu=npu[npu<=11]
print(npu)
n=random.randint(0, len(npu))

In [ ]:
def transformtoinput(impath,imname,imsavepath):
    img = cv2.imread(impath+imname)
#     showimg(img,"nn")
    c1,c2,c3 = cv2.split(img)
    conds = [c2==0,c2==1,c2==2,c2==3,c2==4,c2==5,c2==6,c2==7,c2==8,c2==9,c2==10,c2==11,c2==12,c2==13,c2==14,c2==15,c2==16,c2==17]
    coice = [1,2,3,4,5,6,7,8,9,10,11,12,127,255,127,60,127,60]
    mb=np.select(conds,coice,default=0)
    npu=np.unique(mb)
    npu=list(npu[npu<=12])
    n=random.randint(1, len(npu))
    selectedmasks=random.sample(npu, n)
    gtmap = np.zeros((256,256,3), np.uint8)
    for k,v in colmap.items():
        gtmap[np.where(mb==k)]=v
        
    for k in selectedmasks:
        gtmap[np.where(mb==k)]=(0,0,0)
#     showimg(mb,"plot")
    cv2.imwrite(imsavepath+imname,gtmap)

In [ ]:
# impath="/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/floorplan_dataset/"
# imsavepathinput="/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/train/input/"
impath="PATH_TO_RPLAN_DATASET"
imsavepathinput = "PATH_TO_SAVE_DIR"

In [ ]:
for i in range(len(imglst)):
    transformtoinput(impath,imglst[i],imsavepathinput)

# Splitting Data

In [38]:
import os, os.path

imgs_names = []
path = "/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/training/input/"
for f in os.listdir(path):
    imgs_names.append(f)

In [39]:
test_names = sample(imgs_names,5000)

for name in test_names:
    imgs_names.remove(name)

In [40]:
validate_names = sample(imgs_names,5000)

for name in validate_names:
    imgs_names.remove(name)

In [41]:
train_names = imgs_names

In [42]:
pick_input_images_from = '/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/training/input/'
test_input = '/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/test/input/'
validate_input = '/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/validate/input/'
train_input = '/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/train/input/'

In [47]:
pick_target_images_from = '/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/training/target/'
test_target = '/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/test/target/'
validate_target = '/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/validate/target/'
train_target = '/Users/hardikasnani/Documents/Masters/Spring 2022/Project/Computer Vision/gan/data/train/target/'

In [50]:
os.chdir(pick_target_images_from)
dst_dir = train_input
for f in os.listdir():
    if f in train_names:
        shutil.copy(f, dst_dir)